# **Leetcode - SQL**
181.Find employees who earn more than their managers

In [3]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import SparkSession

In [5]:
empSchema = StructType([
    StructField('id', IntegerType(), True),
    StructField('name', StringType(), True),
    StructField('salary', IntegerType(), True),
    StructField("managerid", IntegerType(), True)
                        ])

In [6]:
emp =[
    (1, 'Joe', 7000, 3),
    (2, 'Henry',8000, 4),
    (3, 'Sam',6000, None),
    (4, 'Max', 9000, None)
]

In [7]:
ss = SparkSession.builder.appName('employee').getOrCreate()
emp_df = ss.createDataFrame(emp, empSchema)
emp_df.show()

In [20]:
rdf=emp_df.alias('e1').join(emp_df.alias('e2'),col('e1.managerid')==col('e2.id'))
#rdf.show()
rdf=rdf.filter(col('e1.salary')>col('e2.salary'))
rdf.show()
rdf.select(col('e1.name').alias('Employee')).show()

+---+----+------+---------+---+----+------+---------+
| id|name|salary|managerid| id|name|salary|managerid|
+---+----+------+---------+---+----+------+---------+
|  1| Joe|  7000|        3|  3| Sam|  6000|     NULL|
+---+----+------+---------+---+----+------+---------+

+--------+
|Employee|
+--------+
|     Joe|
+--------+



LC-182 Duplicate Emails

In [29]:
# This is pyspark implementation of LC-182

from pyspark.sql.functions import *
from pyspark.sql import *
from pyspark.sql.types import *

ss = SparkSession.builder.appName('email').getOrCreate()

emailSchema = StructType([
    StructField('id', IntegerType(), True),
    StructField('email', StringType(),True)
])

emails = [
    (1, 'a@gmail.com'),
    (2, 'b@gmail.com'),
    (3, 'a@gmail.com')
]

email_df = ss.createDataFrame(emails, emailSchema)
email_df.show()

rdf = email_df.groupBy(col('email'))
rdf = rdf.agg(count(col('id')).alias('count')).filter(col('count')>1)
rdf.select(col('email')).show()

+---+-----------+
| id|      email|
+---+-----------+
|  1|a@gmail.com|
|  2|b@gmail.com|
|  3|a@gmail.com|
+---+-----------+

+-----------+
|      email|
+-----------+
|a@gmail.com|
+-----------+



LC-183 Customers Who never Order

In [33]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

ss = SparkSession.builder.appName('customers').getOrCreate()

custSchema = StructType([
    StructField('id', IntegerType(), True),
    StructField('name', StringType(), True)
])

orderSchema = StructType([
    StructField('id', IntegerType(), True),
    StructField('customerId', IntegerType(), True)
])

cust = [
    (1, 'Joe'),
    (2, 'Henry'),
    (3, 'Max'),
    (4, 'Sam')
]

order = [
    (1, 3),
    (2, 1)
]

cust_df = ss.createDataFrame(cust, custSchema)
order_df = ss.createDataFrame(order, orderSchema)

rdf = cust_df.alias('c').join(order_df.alias('o'), col('c.id')==col('o.customerId'), 'left')
rdf.show()
rdf = rdf.filter(col('o.id').isNull())
rdf.select(col('name').alias('customers')).show()

+---+-----+----+----------+
| id| name|  id|customerId|
+---+-----+----+----------+
|  1|  Joe|   2|         1|
|  2|Henry|NULL|      NULL|
|  3|  Max|   1|         3|
|  4|  Sam|NULL|      NULL|
+---+-----+----+----------+

+---------+
|customers|
+---------+
|    Henry|
|      Sam|
+---------+



LC-196 Delete Duplicate Emails

In [35]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

ss = SparkSession.builder.appName('duplicate_emails').getOrCreate()

email = [
    (1, 'a@gmail.com'),
    (2, 'b@gmail.com'),
    (3, 'a@gmail.com')
]

'''
here the expected output is
(1, a@gmail.com)
(2, b@gmail.com)
we only fetch the record with min id and delete all other duplicates of this record
'''

emailSchema = StructType([
    StructField('id', IntegerType(), True),
    StructField('email', StringType(), True)
])

email_df = ss.createDataFrame(email, emailSchema)
email_df.show()

+---+-----------+
| id|      email|
+---+-----------+
|  1|a@gmail.com|
|  2|b@gmail.com|
|  3|a@gmail.com|
+---+-----------+



In [47]:
rdf = email_df.alias('e1').join(email_df.alias('e2'),col('e1.email')==col('e2.email'), 'left')
rdf.show()
rdf = rdf.filter(col('e1.id')>col('e2.id')).select('e1.id', 'e1.email')
rdf.show()
result = email_df.join(rdf, (email_df.id == rdf.id) & (email_df.email == rdf.email), 'left_anti')
result.show()

+---+-----------+---+-----------+
| id|      email| id|      email|
+---+-----------+---+-----------+
|  1|a@gmail.com|  3|a@gmail.com|
|  1|a@gmail.com|  1|a@gmail.com|
|  3|a@gmail.com|  3|a@gmail.com|
|  3|a@gmail.com|  1|a@gmail.com|
|  2|b@gmail.com|  2|b@gmail.com|
+---+-----------+---+-----------+

+---+-----------+
| id|      email|
+---+-----------+
|  3|a@gmail.com|
+---+-----------+

+---+-----------+
| id|      email|
+---+-----------+
|  1|a@gmail.com|
|  2|b@gmail.com|
+---+-----------+

